# Clustering

In this assignment, you will implement a K-Means Clustering algorithm from scratch and compare the results to existing sklearn algorithm.

In [138]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans

Question 1.1: Write a method that determine Labels from Points and ClusterCentroids, and return a list of a label for each point

In [139]:
def euclidean_distance(x1, x2):
    return np.sqrt(np.sum(x1-x2)**2)

# create a function that calculates the Minkowski distance
def minkowski_distance(x1, x2, p):
    return np.sum(np.abs(x1-x2)**p)**(1/p)

def FindLabelOfClosest(Points, ClusterCentroids): # determine Labels from Points and ClusterCentroids
    NumberOfClusters, NumberOfDimensions = ClusterCentroids.shape # dimensions of the initial Centroids
    Distances = np.zeros(NumberOfClusters) # array to store distances
    NumberOfPoints, NumberOfDimensions = Points.shape
    Labels = np.zeros(NumberOfPoints) # array to store labels
    for PointNumber in range(NumberOfPoints): # assign labels to all data points            
        for ClusterNumber in range(NumberOfClusters): # for each cluster
            # Get distances for each cluster
            Distances[ClusterNumber] = euclidean_distance(Points.loc[PointNumber, :], ClusterCentroids.loc[ClusterNumber, :])
        Labels[PointNumber] = np.argmin(Distances)
    return Labels # return the a label for each point


Question 1.2: Write a method that determine centroid of Points with the same label

In [140]:
def CalculateClusterCentroid(Points, Labels): # determine centroid of Points with the same label
    ClusterLabels = np.unique(Labels) # names of labels
    NumberOfPoints, NumberOfDimensions = Points.shape
    ClusterCentroids = pd.DataFrame(np.array([[float('nan')]*NumberOfDimensions]*len(ClusterLabels)))
    for ClusterNumber in ClusterLabels: # for each cluster
        # get mean for each label 
        ClusterCentroids.loc[ClusterNumber, :] = np.mean(Points.loc[ClusterNumber == Labels, :])
    return ClusterCentroids # return the a label for each point

Question 1.3: Put it all together as such. K-means algorithm partitions the input data into K clusters by iterating between the following two steps:
- Compute the cluster center by computing the arithmetic mean of all the points belonging to the cluster.
- Assign each point to the closest cluster center.

In [141]:
def KMeans(Points, ClusterCentroidGuesses):
    ClusterCentroids = ClusterCentroidGuesses.copy()
    Labels_Previous = None
    # Get starting set of labels
    Labels = FindLabelOfClosest(Points, ClusterCentroids)
    i = 0
    while not np.array_equal(Labels, Labels_Previous):
        print(f'Iteration {i}')
        i += 1
        # Re-calculate cluster centers based on new set of labels
        ClusterCentroids = CalculateClusterCentroid(Points, Labels)
        Labels_Previous = Labels.copy() # Must make a deep copy
        # Determine new labels based on new cluster centers
        Labels = FindLabelOfClosest(Points, ClusterCentroids)
    return Labels, ClusterCentroids


In [142]:
StoreTxn = pd.read_csv("Superstore Transaction data.csv")
StoreTxn['Order Date'] = pd.to_datetime(StoreTxn['Order Date'] )
#print(StoreTxn['Order Date'].dtype)
print(StoreTxn.shape)
StoreTxn.head()


(9994, 21)


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,2016-11-08,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,2016-06-12,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,2015-10-11,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,2015-10-11,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [143]:
# Make the datatime fields actual datetime objects
StoreTxn['Ship Date'] = pd.to_datetime(StoreTxn['Ship Date'])
StoreTxn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Row ID         9994 non-null   int64         
 1   Order ID       9994 non-null   object        
 2   Order Date     9994 non-null   datetime64[ns]
 3   Ship Date      9994 non-null   datetime64[ns]
 4   Ship Mode      9994 non-null   object        
 5   Customer ID    9994 non-null   object        
 6   Customer Name  9994 non-null   object        
 7   Segment        9994 non-null   object        
 8   Country        9994 non-null   object        
 9   City           9994 non-null   object        
 10  State          9994 non-null   object        
 11  Postal Code    9994 non-null   int64         
 12  Region         9994 non-null   object        
 13  Product ID     9994 non-null   object        
 14  Category       9994 non-null   object        
 15  Sub-Category   9994 n

Extract RFM features from the transaction data:
- Recency: when was the last purchase they made
- Frequency: how often do they make a purchase in the last month (or any given window you choose)
- Monetary: how much money did they spend in the last month

Question 2.1:
- Use groupby to summarize the quantity and dollar columns by user_id and date
- Name the aggregated data txn_agg
- Reset the index for txn_agg to the default and user_id and date to dataframe columns
- Confirm changes

In [144]:
txn_agg = StoreTxn.groupby(['Customer ID', 'Order Date']).agg({'Sales': 'sum', 'Quantity': 'sum'}).reset_index()
print(txn_agg.shape)
txn_agg.head()

(4992, 4)


,Customer ID,Order Date,Sales,Quantity
0,AA-10315,2014-03-31,726.548,4
1,AA-10315,2014-09-15,29.500,5
2,AA-10315,2015-10-04,26.960,2
3,AA-10315,2016-03-03,4406.072,14
4,AA-10315,2017-06-29,374.480,5


Question 2.2:Using the aggregated data, obtain recency, frequency and monetary features for both dollar and quantity. Use a 7-day moving window for frequency and monetary. Call your new features last_visit_ndays (recency) quantity_roll_sum_7D (frequency) and dollar_roll_sum_7D (monetary)

In [145]:
last = txn_agg.copy()
# I'm using the number of days of the current purchase prior to the most recent purchase as a proxy for recency
last['last_visit_ndays'] = (last['Order Date'].max() - last['Order Date']).dt.days
last = last.drop(['Sales', 'Quantity', 'Order Date'], axis=1)
print(last.head(10), end='\n\n')

roll = txn_agg.copy()
roll['Quantity'] = (roll.groupby('Customer ID', as_index = False, group_keys = False)
                    .apply(lambda g: g.rolling('7D', on = 'Order Date')['Quantity'].sum(), include_groups=False))

roll['Sales'] = (roll.groupby('Customer ID', as_index = False, group_keys = False)
                    .apply(lambda g: g.rolling('7D', on = 'Order Date')['Sales'].sum(), include_groups=False))

roll.rename(columns = {'Quantity' : 'Quantity_roll_sum_7D', 'Sales' : 'Sales_roll_sum_7D'}, inplace = True) # Name the resulting data values quantity_roll_sum_7D and dollar_roll_sum_7D

roll = roll.drop(['Order Date'], axis=1)
print(roll.head(10), end='\n\n')


  Customer ID  last_visit_ndays
0    AA-10315              1370
1    AA-10315              1202
2    AA-10315               818
3    AA-10315               667
4    AA-10315               184
5    AA-10375              1349
6    AA-10375              1163
7    AA-10375              1061
8    AA-10375               967
9    AA-10375               778

  Customer ID  Sales_roll_sum_7D  Quantity_roll_sum_7D
0    AA-10315            726.548                   4.0
1    AA-10315             29.500                   5.0
2    AA-10315             26.960                   2.0
3    AA-10315           4406.072                  14.0
4    AA-10315            374.480                   5.0
5    AA-10375             16.520                   5.0
6    AA-10375             34.272                   3.0
7    AA-10375            178.370                   5.0
8    AA-10375              5.248                   2.0
9    AA-10375             84.960                   6.0



Question 2.3: Combine all three features into a single DataFrame and call it txn_roll

In [146]:
# Inner join between roll (frequency and monetary fields) and last (recency fields) 
# to create churn_roll.  Join based on index which works given that both 
# dataframes are sorted by user_id and date.
churn_roll = pd.merge(last, roll, how='inner')

print(churn_roll.dtypes, end='\n\n')
churn_roll.head(10)


Customer ID              object
last_visit_ndays          int64
Sales_roll_sum_7D       float64
Quantity_roll_sum_7D    float64
dtype: object



,Customer ID,last_visit_ndays,Sales_roll_sum_7D,Quantity_roll_sum_7D
0,AA-10315,1370,726.548,4.0
1,AA-10315,1370,29.500,5.0
2,AA-10315,1370,26.960,2.0
3,AA-10315,1370,4406.072,14.0
4,AA-10315,1370,374.480,5.0
5,AA-10315,1202,726.548,4.0
6,AA-10315,1202,29.500,5.0
7,AA-10315,1202,26.960,2.0
8,AA-10315,1202,4406.072,14.0
9,AA-10315,1202,374.480,5.0


Question 2.4: Use fillna to replace missing values for recency with a large value like 100 days (whatever makes business sense). HINT: You can use pd.Timedelta('100 days') to set the value.

In [147]:
churn_roll['last_visit_ndays'] = churn_roll['last_visit_ndays'].fillna(pd.Timedelta(days=100))
churn_roll.head(10)

,Customer ID,last_visit_ndays,Sales_roll_sum_7D,Quantity_roll_sum_7D
0,AA-10315,1370,726.548,4.0
1,AA-10315,1370,29.500,5.0
2,AA-10315,1370,26.960,2.0
3,AA-10315,1370,4406.072,14.0
4,AA-10315,1370,374.480,5.0
5,AA-10315,1202,726.548,4.0
6,AA-10315,1202,29.500,5.0
7,AA-10315,1202,26.960,2.0
8,AA-10315,1202,4406.072,14.0
9,AA-10315,1202,374.480,5.0


Question 2.5: Merge the aggregated data churn_agg with the RFM features in churn_roll. You can use the merge method to do this with the right keys specified.

In [148]:
# Merge on Customer ID
txn_rfm = pd.merge(txn_agg, churn_roll, how='inner', on='Customer ID')
txn_rfm.head(10)

,Customer ID,Order Date,Sales,Quantity,last_visit_ndays,Sales_roll_sum_7D,Quantity_roll_sum_7D
0,AA-10315,2014-03-31,726.548,4,1370,726.548,4.0
1,AA-10315,2014-03-31,726.548,4,1370,29.500,5.0
2,AA-10315,2014-03-31,726.548,4,1370,26.960,2.0
3,AA-10315,2014-03-31,726.548,4,1370,4406.072,14.0
4,AA-10315,2014-03-31,726.548,4,1370,374.480,5.0
5,AA-10315,2014-03-31,726.548,4,1202,726.548,4.0
6,AA-10315,2014-03-31,726.548,4,1202,29.500,5.0
7,AA-10315,2014-03-31,726.548,4,1202,26.960,2.0
8,AA-10315,2014-03-31,726.548,4,1202,4406.072,14.0
9,AA-10315,2014-03-31,726.548,4,1202,374.480,5.0


Question 3.1: Train the k-means algorithm you developed earlier on the RFM features using  𝑘=4 . What are the cluster centroids? The cluster centroids should be reported in the original scale, not the standardized scale.

In [133]:
# drop columns that are causing problems... don't know why these were still in here at this point
# but I was getting errors if I didn't drop these
temp = txn_rfm.copy()
temp = temp.drop(['Order Date', 'Sales', 'Quantity', 'Customer ID'], axis=1)

ClusterCentroidGuesses = temp.sample(4)
print(ClusterCentroidGuesses.head(10), end='\n\n')
Labels, ClusterCentroids = KMeans(temp, ClusterCentroidGuesses)

        last_visit_ndays  Sales_roll_sum_7D  Quantity_roll_sum_7D
245280                76           1065.840                   8.0
94476                222             94.380                  12.0
194805                11            153.824                  11.0
146413              1352             11.050                   5.0



KeyError: 0

Question 3.2: Pick few pairs and plot scatter plots along with cluster centroids.

In [ ]:
# I'm sorry I was unable to finish this cell, I don't think I performed the data preparation properly

[Bonus] Question 4: Train k-means model using sklearn library and compare results to the model developed above.

In [137]:
model = KMeans(n_clusters=4)
model.fit(temp)
pd.DataFrame(model.cluster_centers_, columns = [temp.columns])

C:\Users\louis\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


,last_visit_ndays,Sales_roll_sum_7D,Quantity_roll_sum_7D
0,284.909074,229.742380,6.978866
1,597.855076,5624.574657,17.705882
2,1020.208954,225.333366,6.964265
3,592.997204,1704.329202,14.593114


Question 5: Create a new text cell in your Notebook: Complete a 50-100 word summary (or short description of your thinking in applying this week's learning to the solution) of your experience in this assignment. Include: What was your incoming experience with this model, if any? what steps you took, what obstacles you encountered. how you link this exercise to real-world, machine learning problem-solving. (What steps were missing? What else do you need to learn?) This summary allows your instructor to know how you are doing and allot points for your effort in thinking and planning, and making connections to real-world work.

I did not spend a lot of time on actually implementing the Kmeans functions. However I found the steps for computing RFM metrics to be very confusing. I also didn't find any examples in our text book. I did run across some examples for doing RFM (on the same dataset in fact). However, even those instructions didn't exactly line up with how we needed to have our data prepared for the KMeans algorithm.

Unfortunately, I don't think I correctly prepared the data before I gave the data to the Kmeans algorithm. My intent was to try a couple different distance metrics (Euclidean and Minkowski) to compare results using each distance metric. But I didn't get far enough with my data prep to be successful. I really look forward to seeing a solution for this one, because this is the first assignment that I have been unable to complete.